In [1]:
import json
from pymongo import MongoClient

In [71]:
client = MongoClient("localhost", 27017)
db = client["CM_data"]

In [3]:
collection = db["scrapped_books"]
client = MongoClient("localhost", 27017)

In [25]:
with open("book_list_fromDF.json", "r") as library:    
    data = json.load(library)

In [26]:
def chunk_data(data, chunk_size):
    for i in range(0, len(data), chunk_size):
        yield data[i:i+chunk_size]

In [27]:
chuck_size = 100 #у нас всего 1000 книг
data_chunks = list(chunk_data(data, chuck_size))

In [28]:
for chunk in data_chunks:
    collection.insert_many(chunk)

print("Данные загружены.")

Данные успешно вставлены.


In [72]:
all_docs = collection.find()
first_doc = all_docs[0]

In [73]:
first_doc

{'_id': ObjectId('65b3b0d41d3196a72d473e23'),
 'Title': "It's Only the Himalayas",
 'Price, £': 45.17,
 'In stock': 19,
 'Description': '“Wherever you go, whatever you do, just . . . don’t do anything stupid.” —My MotherDuring her yearlong adventure backpacking from South Africa to Singapore, S. Bedford definitely did a few things her mother might classify as "stupid." She swam with great white sharks in South Africa, ran from lions in Zimbabwe, climbed a Himalayan mountain without training in Nepal, and wa “Wherever you go, whatever you do, just . . . don’t do anything stupid.” —My MotherDuring her yearlong adventure backpacking from South Africa to Singapore, S. Bedford definitely did a few things her mother might classify as "stupid." She swam with great white sharks in South Africa, ran from lions in Zimbabwe, climbed a Himalayan mountain without training in Nepal, and watched as her friend was attacked by a monkey in Indonesia.But interspersed in those slightly more crazy moments,

In [32]:
pretty_json = json.dumps(first_doc, indent=2, default=str)
print(pretty_json)

{
  "_id": "65b3b0d41d3196a72d473e23",
  "Title": "It's Only the Himalayas",
  "Price, \u00a3": 45.17,
  "In stock": 19,
  "Description": "\u201cWherever you go, whatever you do, just . . . don\u2019t do anything stupid.\u201d \u2014My MotherDuring her yearlong adventure backpacking from South Africa to Singapore, S. Bedford definitely did a few things her mother might classify as \"stupid.\" She swam with great white sharks in South Africa, ran from lions in Zimbabwe, climbed a Himalayan mountain without training in Nepal, and wa \u201cWherever you go, whatever you do, just . . . don\u2019t do anything stupid.\u201d \u2014My MotherDuring her yearlong adventure backpacking from South Africa to Singapore, S. Bedford definitely did a few things her mother might classify as \"stupid.\" She swam with great white sharks in South Africa, ran from lions in Zimbabwe, climbed a Himalayan mountain without training in Nepal, and watched as her friend was attacked by a monkey in Indonesia.But inte

In [33]:
# Получение количества документов в коллекции с помощью функции count_documents()
count = collection.count_documents({})
print(f'Число записей в базе данных: {count}')

Число записей в базе данных: 1000


In [36]:
# фильтрация документов по критериям: количество на остатках > 5 единиц
query = {"In stock": {"$gt": 5}}
print(f"Количество наименований с остатком больше 5: {collection.count_documents(query)}")

Количество наименований с остатком больше 5: 580


In [48]:
# Использование проекции - выведем наименование и цену по книгам с остатком > 15
query = {"In stock": {"$gt": 15}}
projection = {"Title": 1, "Price, \u00a3": 1, "_id": 0}
proj_docs = collection.find(query, projection)
for doc in proj_docs:
    print(doc)

{'Title': "It's Only the Himalayas", 'Price, £': 45.17}
{'Title': 'Sharp Objects', 'Price, £': 47.82}
{'Title': 'In a Dark, Dark Wood', 'Price, £': 19.63}
{'Title': 'The Past Never Ends', 'Price, £': 56.5}
{'Title': 'A Murder in Time', 'Price, £': 16.64}
{'Title': 'Tipping the Velvet', 'Price, £': 53.74}
{'Title': 'Forever and Forever: The Courtship of Henry Longfellow and Fanny Appleton', 'Price, £': 29.69}
{'Title': 'A Flight of Arrows (The Pathfinders #2)', 'Price, £': 55.53}
{'Title': "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)", 'Price, £': 52.29}
{'Title': 'Tsubasa: WoRLD CHRoNiCLE 2 (Tsubasa WoRLD CHRoNiCLE #2)', 'Price, £': 16.28}
{'Title': 'This One Summer', 'Price, £': 19.49}
{'Title': 'The Nameless City (The Nameless City #1)', 'Price, £': 38.16}
{'Title': 'Saga, Volume 5 (Saga (Collected Editions) #5)', 'Price, £': 51.04}
{'Title': 'Rat Queens, Vol. 3: Demons (Rat Queens (Collected Editions) #11-15)', 'Price, £': 50.4}
{'Title': 'Princess Jellyfish 2-in-1 Omnib

In [56]:
# Использование оператора $lt и $gte
query = {"Price, \u00a3": {"$lt": 20}}
print(f"Количество книг по цене < \u00a320: {collection.count_documents(query)}")
query = {
    "$and":
    [
        {"Price, \u00a3": {"$gte": 20},
         "Price, \u00a3": {"$lt": 25}}
    ]
}
print(f"Количество книг по цене >= \u00a320 и < \u00a325: {collection.count_documents(query)}")

Количество книг по цене < £20: 196
Количество книг по цене >= £20 и < £25: 299


In [63]:
# Использование оператора $regex
query = {"Title": {"$regex": "history"}}
print(f"Количество книг, содержащих 'history' в названии (case-sensitive): {collection.count_documents(query)}")
query = {"Title": {"$regex": "history", "$options": "i"}}
print(f"Количество книг, содержащих 'history' в названии (case-insensitive): {collection.count_documents(query)}")

Количество книг, содержащих 'history' в названии (case-sensitive): 0
Количество книг, содержащих 'history' в названии (case-insensitive): 16


In [64]:
# Использование оператора $in
query = {"Category": {"$in": ["Travel", "Mystery"]}}
print(f"Количество книг в категориях 'Travel', 'Mystery': {collection.count_documents(query)}")

Количество книг в категориях 'Travel', 'Mystery': 43


In [70]:
# Использование оператора $ne
query = {"Price, \u00a3" : {"$ne": 33.29}}
print(f"Количество книг по цене не равной \u00a333.29: {collection.count_documents(query)}")

Количество книг по цене не равной £33.29: 999
